# **HW1: Regression** 
In *assignment 1*, you need to finish:

1.  Basic Part: Implement the regression model to predict the number of dengue cases


> *   Step 1: Split Data
> *   Step 2: Preprocess Data
> *   Step 3: Implement Regression
> *   Step 4: Make Prediction
> *   Step 5: Train Model and Generate Result

2.  Advanced Part: Implementing a regression model to predict the number of dengue cases in a different way than the basic part

In [648]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. Basic Part (60%)
In the first part, you need to implement the regression to predict the number of dengue cases

Please save the prediction result in a csv file **hw1_basic.csv**


## Import Packages

> Note: You **cannot** import any other package in the basic part

In [649]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math
import random

## Global attributes
Define the global attributes

In [650]:
#input_dataroot = '/content/drive/MyDrive/ML_HW/HW1/hw1_basic_input.csv' # Input file named as 'hw1_basic_input.csv'
input_dataroot = 'hw1_basic_input.csv'
output_dataroot = 'hw1_basic.csv' # Output file will be named as 'hw1_basic.csv'
output_dataroot_advanced = 'hw1_advanced.csv' 

input_datalist =  [] # Initial datalist, saved as numpy array
output_datalist =  [] # Your prediction, should be 10 * 4 matrix and saved as numpy array
             # The format of each row should be ['epiweek', 'CityA', 'CityB', 'CityC']

You can add your own global attributes here


In [651]:
training_data = []
validation_data = []
T_a = []
T_b = []
T_c = []
C_a = []
C_b = []
C_c = []
W_a = []
W_b = []
W_c = []

## Load the Input File
First, load the basic input file **hw1_basic_input.csv**

Input data would be stored in *input_datalist*

In [652]:
# Read input csv to datalist
with open(input_dataroot, newline='') as csvfile:
  input_datalist = np.array(list(csv.reader(csvfile)))

## Implement the Regression Model

> Note: It is recommended to use the functions we defined, you can also define your own functions


### Step 1: Split Data
Split data in *input_datalist* into training dataset and validation dataset 



In [653]:
def SplitData():
  global training_data, validation_data
  training_data = input_datalist[1:95, 0:8]
  validation_data = input_datalist[95:105, 0:8]


### Step 2: Preprocess Data
Handle the unreasonable data
> Hint: Outlier and missing data can be handled by removing the data or adding the values with the help of statistics  

In [654]:
def PreprocessData():
  global training_data, validation_data, T_a, T_b, T_c, C_a, C_b, C_c
  delete_idx = []
  delete_idx_v = []
  for idx, data in enumerate(training_data):
    for i, element in enumerate(data):
      if element == '':
        delete_idx.append(idx)
        break
      
      else:
        number = float(element)
        if i == 1:
          if number > 35 or number < 15:
            delete_idx.append(idx)
            break
        elif i == 2:
          if number > 35 or number < 15:
            delete_idx.append(idx)
            break
        elif i == 3:
          if number > 35 or number < 15:
            delete_idx.append(idx)
            break
      
  training_data = np.delete(training_data, delete_idx, 0)

  # for test
  for idx, data in enumerate(validation_data):
    for i, element in enumerate(data):
      if element == '':
        delete_idx_v.append(idx)
        break
      else:
        number = float(element)
        if i == 1:
          if number > 35 or number < 15:
            delete_idx_v.append(idx)
            break
        elif i == 2:
          if number > 35 or number < 15:
            delete_idx_v.append(idx)
            break
        elif i == 3:
          if number > 35 or number < 15:
            delete_idx_v.append(idx)
            break
  validation_data = np.delete(validation_data, delete_idx_v, 0)


  T_a = training_data[:, 1].astype(np.float64)
  T_b = training_data[:, 2].astype(np.float64)
  T_c = training_data[:, 3].astype(np.float64)
  C_a = training_data[:, 4].astype(np.float64)
  C_b = training_data[:, 5].astype(np.float64)
  C_c = training_data[:, 6].astype(np.float64)

In [655]:
def basis_fun(x, power):
  return  x**power + 0

### Step 3: Implement Regression
> Hint: You can use Matrix Inversion, or Gradient Descent to finish this part




In [656]:
def Regression():
  global T_a, T_b, T_c, C_a, C_b, C_c, W_a, W_b, W_c
  phi_a = [[1, T_a[0], T_a[0]**2, C_a[0], C_a[0]]]#, T_a[0]**3
  phi_b = [[1, T_b[0], T_b[0]**2, T_b[0]**3, C_b[0], C_b[0]]]#
  phi_c = [[1, T_c[0], T_c[0]**2, T_c[0]**3, C_c[0], C_c[0]]]#
  for i in range(1, len(T_a)):
    if i == 1:
      phi_a = np.append(phi_a, [[1, T_a[i], T_a[i]**2, C_a[0], C_a[0]]], axis = 0)#, T_a[i]**3
    else:
      phi_a = np.append(phi_a, [[1, T_a[i], T_a[i]**2, C_a[i-1], C_a[i-2]]], axis = 0)#, T_a[i]**3
  for i in range(1, len(T_b)):
    if i == 1:
      phi_b = np.append(phi_b, [[1, T_b[i], T_b[i]**2, T_b[i]**3, C_b[0], C_b[0]]], axis = 0)
    else:
      phi_b = np.append(phi_b, [[1, T_b[i], T_b[i]**2, T_b[i]**3, C_b[i-1], C_b[i-2]]], axis = 0)
  for i in range(1, len(T_c)):
    if i == 1:
      phi_c = np.append(phi_c, [[1, T_c[i], T_c[i]**2, T_c[i]**3, C_c[0], C_c[0]]], axis = 0)#
    else:
      phi_c = np.append(phi_c, [[1, T_c[i], T_c[i]**2, T_c[i]**3, C_c[i-1], C_c[i-2]]], axis = 0)#
  
  W_a = np.dot(np.dot(np.linalg.inv(np.dot(phi_a.T, phi_a)), phi_a.T), C_a)
  W_b = np.dot(np.dot(np.linalg.inv(np.dot(phi_b.T, phi_b)), phi_b.T), C_b)
  W_c = np.dot(np.dot(np.linalg.inv(np.dot(phi_c.T, phi_c)), phi_c.T), C_c)

### MAPE function

In [657]:
def MAPE(predict, real):
  mape_ = 0
  for i in range(len(predict)):
    mape_ = mape_ + abs(real[i] - predict[i]) / predict[i]
  mape = (mape_ / len(predict)) * 100
  print(f"MAPE is {mape:.2f} %")

### Step 4: Make Prediction
Make prediction of testing dataset and store the value in *output_datalist*

In [658]:
def MakePrediction():
  global W_a, W_b, W_c, training_data, validation_data, output_datalist
  if len(training_data) > len(validation_data):
    start = len(training_data) - len(validation_data) - 1
  else:
    start = 0
  X_a = validation_data[:, 1].astype(np.float64)
  A_a = validation_data[:, 4].astype(np.float64)
  Y_a = []
  # -30
  for i in range(len(X_a)):
    if i == 0:
      if start == 0:
        Y_a.append(W_a[0] + W_a[1] * X_a[i] + W_a[2] * X_a[i]**2 + W_a[3] * C_a[start] + W_a[4] * C_a[start])# + W_a[3] * X_a[i]**3
      else:
        Y_a.append(W_a[0] + W_a[1] * X_a[i] + W_a[2] * X_a[i]**2 + W_a[3] * C_a[start] + W_a[4] * C_a[start-1])# + W_a[3] * X_a[i]**3
    elif i == 1:
      Y_a.append(W_a[0] + W_a[1] * X_a[i] + W_a[2] * X_a[i]**2 + W_a[3] * Y_a[0] + W_a[4] * C_a[start])# + W_a[3] * X_a[i]**3
    else:
      Y_a.append(W_a[0] + W_a[1] * X_a[i] + W_a[2] * X_a[i]**2 + W_a[3] * Y_a[i-1] + W_a[4] * Y_a[i-2])# + W_a[3] * X_a[i]**3

  X_b = validation_data[:, 2].astype(np.float64)
  A_b = validation_data[:, 5].astype(np.float64)
  Y_b = []
  # -10
  for i in range(len(X_b)):
    if i == 0:
      if start == 0:
        Y_b.append(W_b[0] + W_b[1] * X_b[i] + W_b[2] * X_b[i]**2 + W_b[3] * X_b[i]**3 + W_b[4] * C_b[start] + W_b[5] * C_b[start])
      else:
        Y_b.append(W_b[0] + W_b[1] * X_b[i] + W_b[2] * X_b[i]**2 + W_b[3] * X_b[i]**3 + W_b[4] * C_b[start] + W_b[5] * C_b[start-1])
    elif i == 1:
      Y_b.append(W_b[0] + W_b[1] * X_b[i] + W_b[2] * X_b[i]**2 + W_b[3] * X_b[i]**3 + W_b[4] * Y_b[0] + W_b[5] * C_b[start])#
    else:
      Y_b.append(W_b[0] + W_b[1] * X_b[i] + W_b[2] * X_b[i]**2 + W_b[3] * X_b[i]**3 + W_b[4] * Y_b[i-1] + W_b[5] * Y_b[i-2])#

  X_c = validation_data[:, 3].astype(np.float64)
  A_c = validation_data[:, 6].astype(np.float64)
  Y_c = []
  # +20
  for i in range(len(X_c)):
    if i == 0:
      if start == 0:
        Y_c.append(W_c[0] + W_c[1] * X_c[i] + W_c[2] * X_c[i]**2 + W_c[3] * X_c[i]**3 + W_c[4] * C_c[start] + W_c[5] * C_c[start])#
      else:
        Y_c.append(W_c[0] + W_c[1] * X_c[i] + W_c[2] * X_c[i]**2 + W_c[3] * X_c[i]**3 + W_c[4] * C_c[start] + W_c[5] * C_c[start-1])#
    elif i == 1:
      Y_c.append(W_c[0] + W_c[1] * X_c[i] + W_c[2] * X_c[i]**2 + W_c[3] * X_c[i]**3 + W_c[4] * Y_c[0] + W_c[5] * C_c[start])#
    else:
      Y_c.append(W_c[0] + W_c[1] * X_c[i] + W_c[2] * X_c[i]**2 + W_c[3] * X_c[i]**3 + W_c[4] * Y_c[i-1] + W_c[5] * Y_c[i-2])#3
  output_datalist = []
  for i in range(len(Y_a)):
    output_datalist.append([validation_data[i][0], Y_a[i], Y_b[i], Y_c[i]])
  print(f'Basic part :')
  print(f'coeff model A : {W_a[2]}, {W_a[1]}, {W_a[0]}\nlagging A : {W_a[3]}, {W_a[4]}')
  print(f'coeff model B : {W_b[3]}, {W_b[2]}, {W_b[1], W_b[0]}\nlagging B : {W_b[4]}, {W_b[5]}')
  print(f'coeff model C : {W_c[1]}, {W_c[0]}\nlagging C : {W_c[2]}, {W_c[3]}')    
  '''
  timer = []
  for i in range(len(Y_a)):
    timer.append(i)
  plt.plot(X_a, Y_a, 'ro', X_a, A_a, 'bo')
  plt.show()
  plt.plot(timer, Y_a, 'r--', timer, A_a, 'b--')
  plt.show()
  
  MAPE(Y_a, A_a)
  
  plt.plot(X_b, Y_b, 'ro', X_b, A_b, 'bo')
  plt.show()
  plt.plot(timer, Y_b, 'r--', timer, A_b, 'b--')
  plt.show()
  
  MAPE(Y_b, A_b)

  plt.plot(X_c, Y_c, 'ro', X_c, A_c, 'bo')
  plt.show()
  plt.plot(timer, Y_c, 'r--', timer, A_c, 'b--')
  plt.show()
  
  MAPE(Y_c, A_c)
  '''

### Step 5: Train Model and Generate Result

> Notice: **Remember to output the coefficients of the model here**, otherwise 5 points would be deducted
* If your regression model is *3x^2 + 2x^1 + 1*, your output would be: 
```
3 2 1
```





In [659]:
SplitData()
PreprocessData()
Regression()
MakePrediction()

Basic part :
coeff model A : 0.06685295406908873, -4.009112836978099, 64.52564018219714
lagging A : 0.6160408835587965, 0.217910576037666
coeff model B : -0.04472929235106071, 2.8484647377899135, (-60.596135664810596, 436.7412292276589)
lagging B : 0.5198886080628984, 0.27777903243770696
coeff model C : 45.0116374339955, -398.5445178645359
lagging C : -1.6625637473041515, 0.020190651065504384


## Write the Output File
Write the prediction to output csv
> Format: 'epiweek', 'CityA', 'CityB', 'CityC'

In [660]:
with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  for row in output_datalist:
    writer.writerow(row)

# 2. Advanced Part (35%)
In the second part, you need to implement the regression in a different way than the basic part to help your predictions for the number of dengue cases

We provide you with two files **hw1_advanced_input1.csv** and **hw1_advanced_input2.csv** that can help you in this part

Please save the prediction result in a csv file **hw1_advanced.csv** 


In [661]:
parameter_a = []
parameter_b = []
parameter_c = []
index = []

### advanced 1

In [662]:
def Regression_advanced():
  global C_a, C_b, C_c, parameter_a, parameter_b, parameter_c, index
  for i in range(len(C_a)):
    index.append(i)
  parameter_a = np.polyfit(index, C_a, 4)
  parameter_b = np.polyfit(index, C_b, 2)
  parameter_c = np.polyfit(index, C_c, 4)

In [663]:
def MakePrediction_advanced():
  global C_a, C_b, C_c, parameter_a, parameter_b, parameter_c, validation_data, training_data, output_datalist
  X = []
  for i in range(len(training_data) - len(validation_data), len(training_data)):
    X.append(i)
  
  A_a = validation_data[:, 4].astype(np.float64)
  Y_a = []
  for i in range(len(X)):
    #Y_a.append(parameter_a[0] * X[i]**7 + parameter_a[1] * X[i]**6 + parameter_a[2] * X[i]**5 + parameter_a[3] * X[i]**4 + parameter_a[4] * X[i]**3 + parameter_a[5] * X[i]**2 + parameter_a[6] * X[i] + parameter_a[7])
    #Y_a.append(parameter_a[0] * X[i]**6 + parameter_a[1] * X[i]**5 + parameter_a[2] * X[i]**4 + parameter_a[3] * X[i]**3 + parameter_a[4] * X[i]**2 + parameter_a[5] * X[i]**1 + parameter_a[6])
    #Y_a.append(parameter_a[0] * X[i]**5 + parameter_a[1] * X[i]**4 + parameter_a[2] * X[i]**3 + parameter_a[3] * X[i]**2 + parameter_a[4] * X[i] + parameter_a[5])
    Y_a.append(parameter_a[0] * X[i]**4 + parameter_a[1] * X[i]**3 + parameter_a[2] * X[i]**2 + parameter_a[3] * X[i]+ parameter_a[4])
    #Y_a.append(parameter_a[0] * X[i]**3 + parameter_a[1] * X[i]**2 + parameter_a[2] * X[i] + parameter_a[3])
  
  
  A_b = validation_data[:, 5].astype(np.float64)
  Y_b = []
  for i in range(len(X)):
    #Y_b.append(parameter_b[0] * X[i]**7 + parameter_b[1] * X[i]**6 + parameter_b[2] * X[i]**5 + parameter_b[3] * X[i]**4 + parameter_b[4] * X[i]**3 + parameter_b[5] * X[i]**2 + parameter_b[6] * X[i] + parameter_b[7])
    Y_b.append(parameter_b[0] * X[i]**2 + parameter_b[1] * X[i] + parameter_b[2])
  
  A_c = validation_data[:, 6].astype(np.float64)
  Y_c = []
  for i in range(len(X)):
    #Y_c.append(parameter_c[0] * X[i]**5 + parameter_c[1] * X[i]**4 + parameter_c[2] * X[i]**3 + parameter_c[3] * X[i]**2 + parameter_c[4] * X[i] + parameter_c[5])
    Y_c.append(parameter_c[0] * X[i]**4 + parameter_c[1] * X[i]**3 + parameter_c[2] * X[i]**2 + parameter_c[3] * X[i] + parameter_c[4])
  
  output_datalist = [[ 'epiweek', 'CityA', 'CityB', 'CityC']]
  for i in range(len(Y_a)):
    output_datalist.append([validation_data[i][0], Y_a[i], Y_b[i], Y_c[i]])
  
  timer = []
  for i in range(len(Y_a)):
    timer.append(i)
  
  print(f'Advanced part :')
  print(f'coeff model A : {parameter_a}')
  print(f'coeff model B : {parameter_a}')
  print(f'coeff model C : {parameter_a}')

  plt.plot(timer, Y_a, 'ro', timer, A_a, 'bo')
  plt.show()
  MAPE(Y_a, A_a)

  plt.plot(timer, Y_b, 'ro', timer, A_b, 'bo')
  plt.show()
  MAPE(Y_b, A_b)
  
  plt.plot(timer, Y_c, 'ro', timer, A_c, 'bo')
  plt.show()
  MAPE(Y_c, A_c)

### advanced 2 with past case

In [664]:
def Regression_advanced_phi():
  global C_a, C_b, C_c, parameter_a, parameter_b, parameter_c, index
  for i in range(len(C_a)):
    index.append(i)

  phi_a = [[1, index[0], index[0]**2, index[0]**3, C_a[0], C_a[0]]]
  for i in range(1, len(index)):
    if i == 1:
      phi_a = np.append(phi_a, [[1, index[i], index[i]**2, index[i]**3, C_a[0], C_a[0]]], axis = 0)
    else:
      phi_a = np.append(phi_a, [[1, index[i], index[i]**2, index[i]**3, C_a[i-1], C_a[i-2]]], axis = 0)
  
  phi_b = [[1, index[0], index[0]**2, C_b[0], C_b[0]]]
  for i in range(1, len(index)):
    if i == 1:
      phi_b = np.append(phi_b, [[1, index[i], index[i]**2, C_b[0], C_b[0]]], axis = 0)
    else:
      phi_b = np.append(phi_b, [[1, index[i], index[i]**2, C_b[i-1], C_b[i-2]]], axis = 0)

  phi_c = [[1, index[0], index[0]**2, index[0]**3, index[0]**4, C_c[0], C_c[0]]]
  for i in range(1, len(index)):
    if i == 1:
      phi_c = np.append(phi_c, [[1, index[i], index[i]**2, index[i]**3, index[i]**4, C_c[0], C_c[0]]], axis = 0)
    else:
      phi_c = np.append(phi_c, [[1, index[i], index[i]**2, index[i]**3, index[i]**4, C_c[i-1], C_c[i-2]]], axis = 0)

  parameter_a = np.dot(np.dot(np.linalg.inv(np.dot(phi_a.T, phi_a)), phi_a.T), C_a)
  parameter_b = np.dot(np.dot(np.linalg.inv(np.dot(phi_b.T, phi_b)), phi_b.T), C_b)
  parameter_c = np.dot(np.dot(np.linalg.inv(np.dot(phi_c.T, phi_c)), phi_c.T), C_c)

In [665]:
def MakePrediction_advanced_phi():
  global C_a, C_b, C_c, parameter_a, parameter_b, parameter_c, validation_data, training_data, output_datalist
  X = []
  for i in range(len(training_data) - len(validation_data), len(training_data)):
    X.append(i)

  if len(training_data) > len(validation_data):
    start = len(training_data) - len(validation_data)
  else:
    start = 0
  A_a = validation_data[:, 4].astype(np.float64)
  Y_a = []
  for i in range(len(X)):
    if i == 0:
      if start == 0:
        Y_a.append(parameter_a[3] * X[i]**3 + parameter_a[2] * X[i]**2 + parameter_a[1] * X[i]+ parameter_a[0] + parameter_a[4] * C_a[0] + parameter_a[5] * C_a[0])
      else:
        Y_a.append(parameter_a[3] * X[i]**3 + parameter_a[2] * X[i]**2 + parameter_a[1] * X[i]+ parameter_a[0] + parameter_a[4] * C_a[start] + parameter_a[5] * C_a[start-1])
    elif i == 1:
      Y_a.append(parameter_a[3] * X[i]**3 + parameter_a[2] * X[i]**2 + parameter_a[1] * X[i]+ parameter_a[0] + parameter_a[4] * Y_a[0] + parameter_a[5] * C_a[start])
    else:
      Y_a.append(parameter_a[3] * X[i]**3 + parameter_a[2] * X[i]**2 + parameter_a[1] * X[i]+ parameter_a[0] + parameter_a[4] * Y_a[i-1] + parameter_a[5] * Y_a[i-2])

  A_b = validation_data[:, 5].astype(np.float64)
  Y_b = []
  for i in range(len(X)):
    if i == 0:
      if start == 0:
        Y_b.append(parameter_b[2] * X[i]**2 + parameter_b[1] * X[i]+ parameter_b[0] + parameter_b[3] * C_b[0] + parameter_b[4] * C_b[0])
      else:
        Y_b.append(parameter_b[2] * X[i]**2 + parameter_b[1] * X[i]+ parameter_b[0] + parameter_b[3] * C_b[start] + parameter_b[4] * C_b[start-1])
    elif i == 1:
      Y_b.append(parameter_b[2] * X[i]**2 + parameter_b[1] * X[i]+ parameter_b[0] + parameter_b[3] * Y_b[0] + parameter_b[4] * C_b[start])
    else:
      Y_b.append(parameter_b[2] * X[i]**2 + parameter_b[1] * X[i]+ parameter_b[0] + parameter_b[3] * Y_b[i-1] + parameter_b[4] * Y_b[i-2])

  A_c = validation_data[:, 6].astype(np.float64)
  Y_c = []
  for i in range(len(X)):
    if i == 0:
      if start == 0:
        Y_c.append(parameter_c[4] * X[i]**4 + parameter_c[3] * X[i]**3 + parameter_c[2] * X[i]**2 + parameter_c[1] * X[i]+ parameter_c[0] + parameter_c[5] * C_c[0] + parameter_c[6] * C_c[0])
      else:
        Y_c.append(parameter_c[4] * X[i]**4 + parameter_c[3] * X[i]**3 + parameter_c[2] * X[i]**2 + parameter_c[1] * X[i]+ parameter_c[0] + parameter_c[5] * C_c[start] + parameter_c[6] * C_c[start-1])
    elif i == 1:
      Y_c.append(parameter_c[4] * X[i]**4 + parameter_c[3] * X[i]**3 + parameter_c[2] * X[i]**2 + parameter_c[1] * X[i]+ parameter_c[0] + parameter_c[5] * Y_c[0] + parameter_c[6] * C_c[start])
    else:
      Y_c.append(parameter_c[4] * X[i]**4 + parameter_c[3] * X[i]**3 + parameter_c[2] * X[i]**2 + parameter_c[1] * X[i]+ parameter_c[0] + parameter_c[5] * Y_c[i-1] + parameter_c[6] * Y_c[i-2])
 
  timer = []
  for i in range(len(Y_a)):
    timer.append(i)
  
  print(f'Advanced part :')
  print(f'coeff model A : {parameter_a[3]}, {parameter_a[2]}, {parameter_a[1]}, {parameter_a[0]}\nlagging A : {parameter_a[4]}, {parameter_a[5]}')
  print(f'coeff model B : {parameter_b[2]}, {parameter_b[1]}, {parameter_b[0]}\nlagging B : {parameter_b[3]}, {parameter_b[4]}')
  print(f'coeff model C : {parameter_c[4]}, {parameter_c[3]}, {parameter_c[2]}, {parameter_c[1]}, {parameter_c[0]}\nlagging A : {parameter_c[5]}, {parameter_c[6]}')
  
  output_datalist = []
  for i in range(len(Y_a)):
    output_datalist.append([validation_data[i][0], Y_a[i], Y_b[i], Y_c[i]])
  '''
  plt.plot(timer, Y_a, 'r--', timer, A_a, 'b--')
  plt.show()
  MAPE(Y_a, A_a)

  plt.plot(timer, Y_b, 'r--', timer, A_b, 'b--')
  plt.show()
  MAPE(Y_b, A_b)

  plt.plot(timer, Y_c, 'r--', timer, A_c, 'b--')
  plt.show()
  MAPE(Y_c, A_c)
  '''
  

### implement

In [666]:
Regression_advanced_phi()
MakePrediction_advanced_phi()

with open(output_dataroot_advanced, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  for row in output_datalist:
    writer.writerow(row)

Advanced part :
coeff model A : -0.00029330936035904323, 0.037719565492276196, -1.553774956520819, 31.909984687498333
lagging A : 0.5588152664549507, 0.14909509277788563
coeff model B : 0.010985200774266058, -1.1766303441706232, 48.79489427603011
lagging B : 0.20091516395561948, -0.030027902932624667
coeff model C : -1.635545037138791e-05, 0.0021302687668579784, -0.07925316499852451, 0.9152253931603047, 2.075185157696862
lagging A : 0.863866318235521, -0.21828525547433464


# Report *(5%)*

Report should be submitted as a pdf file **hw1_report.pdf**

*   Briefly describe the difficulty you encountered 
*   Summarize your work and your reflections 
*   No more than one page






# Save the Code File
Please save your code and submit it as an ipynb file! (**hw1.ipynb**)